In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 17.5MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
     |████████████████████████████████| 3.2MB 40.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=144def7c934d44fe1b0ed0a7a36ebb37aa218d49645916bbb82c60fe2ae90316
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import DPRReaderTokenizer, TFDPRReader
import json
import tensorflow as tf

In [4]:
tokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
model = TFDPRReader.from_pretrained('facebook/dpr-reader-single-nq-base', from_pt=True)

All PyTorch model weights were used when initializing TFDPRReader.

All the weights of TFDPRReader were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDPRReader for predictions without further training.


In [5]:
def getPassages(filename):
    with open(filename) as file:
        data = json.load(file)
    passages = list()
    for i in data["vertices"]["paragraphs"]:
      passages.append(i["text"])
    return passages

In [6]:
def getTopics(filename, paras):
    with open(filename) as file:
        data = json.load(file)
    topics = ['' for i in range(len(data["vertices"]["topics"]))]
    edges = data["edges"]["main"]
    vertices = data["vertices"]["topics"]
    count = 0
    titles = [""]*len(paras)
    # print(edges)
    for e in edges:
        if e[1] == "about_concept":
          # print(e)
            topic_id = int(str(e[2])[2:])
            para_id = int(str(e[0])[2:])
            titles[para_id] += ";" + vertices[topic_id]["text"]
            count += 1
    for t in range(len(titles)):
      titles[t] = titles[t][1:]
    return titles
    

In [7]:
paras=  getPassages("handbook_graph.json")
titles = getTopics("handbook_graph.json", paras)
# for i in range(len(paras)):
#   print(titles[i],"-", paras[i])

In [21]:
questions = [
             "How do I calculate cgpa",
             "What is the normal load for UG students",
             "If I fail a course and take it again in the later semester, will my earlier course with F grade be removed from the transcript",
            " what is the process of registration?",
            "how many seats are there in cse for admission?",
             " what is the admission criteria for btech",
             "I am in 1st year. Can I take overload?",
             "I am in 2nd year. Can I take overload?",
             "what happens if I miss the endsem because of a medical reason?",
             "what happens if I fail a course?",
             " what happens if I get an F grade in a course?",
             "How can I calculate sgpa",
             "What if I pass all my semesters",
             "What about canteen",
             "Will I get hostel",
             "I dont know anything about IIIT",
             "Who was abraham lincoln",
             "Can i take 8 credits of online courses in a semester",
             "how many credits do i need to graduate",
             "how is my semester graded",
             "what if I do more than 156 credits in my btech course",
             "can I take up internships during a semester?",
              "what is the i grade",
              "can I replace a core course on getting an F grade?",
              "how can I get the grade given to me in a course changed?",
              "how will my cgpa be computed if I do more than 156 credits?",
              "is there any rule for attendance?",
              "how can I apply for a semester leave?",
              "how can I apply for branch transfer from ece to cse",
              "what is the minimum credit requirement for graduation?",
              "what are the requirements to get an honours degree?",
              "when is the convocation held?"
]
for q in questions:
  encoded_input = tokenizer(questions = q, titles = titles, texts = paras, return_tensors = 'tf', padding = True, truncation=True)
  outputs = model(encoded_input)
  relevance_logits = outputs.relevance_logits
  top_answer = tf.math.top_k(relevance_logits, k = 3)[1]
  print("q:", q)
  for i in range(3):
    print("i->", paras[int(top_answer[i])])


q: How do I calculate cgpa
i-> The overall performance is indicated by a Cumulative Grade Point Average (CGPA) which is computed in the same manner as the computation of SGPA but for all the courses done in the program. 
i-> btech students are allowed to do extra credits" beyond 156 credits required for completion of btech requirement. For N extra credits done, the student will be allowed worst grades in N credits, with a maximum limit of at most 8 credits, to be not counted towards CGPA computation. The impact of this decision will be reviewed after two batches have graduated with this option. Following method will be used for calculating the CGPA: 
i-> CGPA computation will remain the same for first 5 semesters, even if a student has done "extra credits." 
q: What is the normal load for UG students
i-> In general, students will take courses as per the program. Normal load for the first and second year students is 16-20 credits and for the third and fourth year students is 16-22 credi

In [11]:
outputs = model(encoded_input)
relevance_logits = outputs.relevance_logits
print(relevance_logits)

tf.Tensor(
[-11.037859  -12.390554  -11.334875  -12.073784   -7.296364  -10.374155
 -10.369619  -11.316698  -11.1159315  -9.898624  -10.500264  -10.312939
 -10.195675   -9.870754   -8.649958  -11.346728  -11.3759    -10.477759
 -10.2176285  -8.346793   -4.004635  -10.82005   -10.794814   -9.724665
  -8.400145  -10.756733  -11.497892  -10.261125  -10.095016   -8.300631
 -11.646077  -11.287343  -11.084473   -9.0968075  -9.602723  -10.229122
  -7.3580933 -10.822889  -11.454759   -9.989601   -3.779549  -10.505032
 -11.340759  -10.492999   -8.622826  -11.218956   -8.8188    -10.380283
  -9.405129  -11.150749   -8.825893   -8.791458  -11.830051  -11.530688
  -8.636441  -11.148133  -11.678968  -10.991806    2.0580568  -1.1651748
  -8.028076  -11.625804   -9.056688   -8.223518   -6.2468596  -7.4776855
  -7.579268   -9.055848   -6.1721277  -8.093563   -2.772825   -8.047393
 -11.224286   -9.80279   -10.461338  -10.920485  -11.088331  -10.341881
 -10.704386  -10.58514   -10.741667   -4.7737756 -1

In [12]:
print(paras[int(tf.math.argmax(relevance_logits))])

The semester performance is indicated by a Semester Grade Point Average (SGPA) which is a weighted sum of all the points earned in the courses done in a semester. The SGPA is given for each semester and is computed using the following formula. The SGPA (u1.w1 + u2.w2 +  + un.wn)  or  (u1 + u2 +  + un) where ui is the number of credits for the course i and wi is the points earned through the letter grade in that course, and n is the number of courses taken in the semester. While doing the computation of SGPA, the course or thesis credits with grades S and X are ignored. 
